In [ ]:
# Modules...
import os
import random
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data import Dataset

import torchio as tio
import nibabel as nib
from scipy.ndimage import label, generate_binary_structure, binary_dilation, binary_erosion

from matplotlib import pyplot as plt
from sklearn.model_selection import KFold, train_test_split, StratifiedKFold

from IPython.display import clear_output
from timm.models.layers import DropPath

In [ ]:
def save_prediction(img, pred, target, output_path, sub_id):
    # Resolve location where data should be written
    if not os.path.exists(output_path):
        raise IOError(
            "Data path, {}, could not be resolved".format(output_path)
        )
            
    # Convert numpy array to NIFTI
    img_nifti = nib.Nifti1Image(img, None)
    pred_nifti = nib.Nifti1Image(pred, None)
    target_nifti = nib.Nifti1Image(target, None)
    
    # Save segmentation to disk
    img_file = "input.nii.gz"
    pred_file = "result.nii.gz"         
    target_file = "target.nii.gz"
    
    nib.save(img_nifti, os.path.join(output_path, img_file))
    nib.save(pred_nifti, os.path.join(output_path, pred_file)) 
    nib.save(target_nifti, os.path.join(output_path, target_file)) 

weights = '/nasdata4/kyj0305/PE_study/son/weights/finetune'
PATH =  weights+'effusion_seg_4fold.pth'
list_path = '/nasdata4/kyj0305/PE_study/son/pretrain_results/cross_4fold.txt'
list = open(list_path, 'r').read().split('\n')
list.remove('')
save_dir='/nasdata4/kyj0305/PE_study/son/pretrain_results/Pretrain/4fold'

#model.load_state_dict(torch.load(PATH, map_location=device))
model.load_state_dict(torch.load(PATH))
model.eval()

test_dsc_list = []

with torch.no_grad():
    for i, test_data in enumerate(loader_test):
        test_images, test_targets = test_data
        test_images = test_images.to(device).float() 
        test_targets = test_targets.to(device).float()
        
        test_outputs = model(test_images) 
        
        test_outputs = torch.where(test_outputs>0.5, 1.0, 0.0)
        dicescore = dice(test_outputs, test_targets)
        test_dsc_list.append(dicescore.item())

        print("[{}]  DICE: {:.3f}".format(list[i], dicescore))
        print("------------------------------------------------------------------------")
        
        test_images = test_images.squeeze()
        test_targets = test_targets.squeeze()
        test_outputs = test_outputs.squeeze()
        test_images = test_images.detach().cpu().numpy()
        test_targets = test_targets.detach().cpu().numpy()
        test_outputs = test_outputs.detach().cpu().numpy()
        
        # save nifti
        subject_ID = list[i]
        output_path = save_dir+'/'+list[i]
        os.mkdir(output_path)
        save_prediction(img=test_images,
                        pred=test_outputs, 
                        target=test_targets, 
                        output_path=output_path,
                        sub_id=subject_ID
                        )

print("평균 BIG   DSC: {:.3f}".format(np.mean(test_dsc_list)))